In [1]:
import requests
from datetime import datetime, timedelta
import pandas as pd 

In [2]:
def usgs_japon_json_etl(url, filename):
    # Establecer el tiempo de inicio desde hoy menos 20000 eventos
    endtime = datetime.today().strftime('%Y-%m-%dT%H:%M:%S')
    starttime = (datetime.today() - timedelta(days=20000)).strftime('%Y-%m-%dT%H:%M:%S')
    
    # Establecer los demás parámetros de solicitud
    params = {"format": "geojson", "starttime": starttime, "endtime": endtime, "minmagnitude": 2.5,
                "minlatitude": 24.40, "maxlatitude": 45.55, "minlongitude": 122.93, "maxlongitude": 153.99, "limit": 20000}
    
    # Realizar la solicitud a la API de USGS
    response = requests.get(url, params=params)
    data = response.json()
    
    # Obtener solo las características (features)
    features = data['features']
    
    # Convertir las características a un DataFrame
    df_sismo = pd.json_normalize(features)
    
    # Seleccionar las columnas necesarias
    df_sismo = df_sismo[["properties.mag", "properties.time", "properties.updated", "properties.tz",
                         "properties.place", "properties.type", "geometry.coordinates"]]
    
    # Transformar las columnas de tiempo
    df_sismo["properties.time"] = pd.to_datetime(df_sismo["properties.time"], unit="ms")
    df_sismo["properties.updated"] = pd.to_datetime(df_sismo["properties.updated"], unit="ms")
    
    # Dividir la columna geometry.coordinates en columnas separadas
    df_sismo[["Longitud", "Latitud", "Profundidad"]] = pd.DataFrame(df_sismo["geometry.coordinates"].tolist())
    
    # Seleccionar las columnas necesarias
    df_sismo = df_sismo[["properties.mag", "properties.time", "properties.updated", "properties.place",
                         "Longitud", "Latitud", "Profundidad"]]
    
    # Renombrar las columnas
    df_sismo = df_sismo.rename(columns={"properties.mag": "Magnitud",
                                        "properties.time": "Primer Registro",
                                        "properties.updated": "Último Registro",
                                        "properties.place": "Ubicación",
                                        "Longitud": "Longitud (grados)",
                                        "Latitud": "Latitud (grados)",
                                        "Profundidad": "Profundidad (km)"})
    
    # Guardar el DataFrame en un archivo JSON
    df_sismo.to_json(path_or_buf= '../JSON/CON_ETL/' +filename, orient="records")
    
    # Devolver el DataFrame
    return df_sismo

In [3]:
url = r"https://earthquake.usgs.gov/fdsnws/event/1/query?"
usgs_japon_json_etl(url, 'japon_etl_prueba1.json')

,Magnitud,Primer Registro,Último Registro,Ubicación,Longitud (grados),Latitud (grados),Profundidad (km)
0,4.4,2023-05-15 03:21:10.118,2023-05-15 03:34:52.040,"102 km WNW of Ie, Japan",126.8405,27.0291,131.397
1,4.9,2023-05-14 22:00:12.432,2023-05-14 22:20:12.040,Kuril Islands,148.4870,45.4160,143.427
2,3.9,2023-05-14 21:27:36.661,2023-05-15 14:17:49.853,"45 km NE of Tonghae, South Korea",129.4615,37.8410,12.353
3,5.1,2023-05-14 16:22:52.585,2023-05-14 16:42:14.040,None,139.1763,33.3737,10.000
4,4.5,2023-05-14 15:24:22.591,2023-05-14 18:13:03.040,"162 km S of Shimoda, Japan",139.2095,33.2302,10.000
...,...,...,...,...,...,...,...
19995,4.8,2009-02-11 05:52:29.880,2014-11-07 01:38:21.636,Kuril Islands,150.1170,45.4630,109.700
19996,3.8,2009-02-08 21:09:22.760,2014-11-07 01:38:21.231,"Izu Islands, Japan region",141.0270,29.1670,17.300
19997,4.7,2009-02-08 00:26:35.310,2014-11-07 01:38:20.965,"127 km NW of Ishikawa, Japan",126.8580,27.1830,25.400
19998,4.5,2009-02-07 03:03:41.410,2014-11-07 01:38:20.767,"81 km ENE of Shikotan, Russia",147.5800,44.2000,62.100
